## Data loading and preprocessing

In [22]:
import pandas as pd
from tqdm import tqdm
import random

In [2]:
df1 = pd.read_csv('without_unknown_train0102.csv')
df2 = pd.read_csv('unknown_labelled.csv')
df2 = df2.drop(columns=['id'])

In [3]:
df1.head()

,data,label
0,Вы как что взяли 3000 рублей? Вас не смущает ч...,0
1,Ищу программиста 1С в г. Липецк\r\n#вакансия #...,1C-разработчик
2,Business analyst в IQVIA\r\n\r\nteletype.in/@d...,Бизнес-аналитик
3,"• Middle Python разработчик в Центробит, Москв...",Python-разработчик
4,Ruby Backend Developer (+Gо)\r\nв Staply — пер...,Ruby-разработчик


In [4]:
df2.head()

,data,label
0,"Team Lead (C#, JavaScript)\nв Тандер, Магнит —...",Несколько вакансий
1,Разработка Telegram ботов любой сложности!\n\n...,0
2,IT-рекрутер в STREAM (ПАО МТС): stream.ru\n\nО...,IT-рекрутер
3,Marketing Assistant в Nestle\n\nteletype.in/@d...,non-IT
4,"• Software Engineer (iOS, Android, React Nativ...",Несколько вакансий


In [5]:
df = pd.concat([df1, df2])

In [6]:
df.sample(5)

,data,label
1138,Ты дурак?),0
3108,#вакансия #qa #python #remote #удаленка \r\n\r...,QA-Инженер
901,Full Stack JavaScript Developer\r\nв Кодабру —...,Full Stack разработчик
882,#вакансия #Москва #iOS #Swift #senior #middle ...,iOS-разработчик
2931,#gamedesigner #designer #remote #удаленно #mid...,non-IT


In [7]:
df['vacancy'] = df['label'].apply(lambda x: 1 if x!='0' else 0)
df['vacancy'].value_counts()

1    2867
0    1119
Name: vacancy, dtype: int64

In [8]:
df['IT'] = df['label'].apply(lambda x: 'non-IT' if (x=='non-IT' or x=='0') else 'IT')
df['IT'].value_counts()

IT        2256
non-IT    1730
Name: IT, dtype: int64

In [9]:
df.sample(5)

,data,label,vacancy,IT
1918,16 года на 16 гиг в М.Видео\r\nAir 😂,0,0,non-IT
2055,#Москва #java #kotlin #backend #spring #middle...,Java-разработчик,1,IT
170,#вакансия #удаленно #remote #офис #МСК #Россия...,Java-разработчик,1,IT
1907,"Привет👋, @VictoriaRomanovna!\r\nСпасибо что оп...",0,0,non-IT
2261,"Английский свободный нужен, тк будут командиро...",0,0,non-IT


## 'Vacancy' filter (featuring spacy)

In [10]:
import spacy
from spacy.tokens import DocBin

In [13]:
df_vacancy = df[['data', 'vacancy']]
df_vacancy.sample(5)

,data,vacancy
312,Product Analyst в Gett\n\nteletype.in/@dreamjo...,1
2667,"Sergey, [07.04.21 00:10]\r\n#вакансия #москва ...",1
1390,#вакансия #удалённо #job #remote #parttime #пр...,1
1977,#ищу таргетолога для продвижения личного блога...,0
1190,Product Manager\r\nв iFarm — российский агроте...,1


In [16]:
vac_train_data = []

for i in range(len(df_vacancy)):
    vac_train_data.append((df_vacancy.iloc[i, 0], df_vacancy.iloc[i, 1]))

In [17]:
def make_docs(data):
    docs = []
    for doc, label in nlp.pipe(data, as_tuples = True):
        if label == 0:
            doc.cats['non-vacancy'] = 1
        else:
            doc.cats['vacancy'] = 1
        docs.append(doc)
    return (docs)

In [26]:
nlp = spacy.load('ru_core_news_sm')

In [24]:
num_texts = int(np.round(len(df_vacancy)*0.8))
num_texts

<IPython.core.display.Javascript object>

3189

In [28]:
train_docs = make_docs(random.choices(vac_train_data, k=num_texts))
doc_bin = DocBin(docs=train_docs)
doc_bin.to_disk("./vacancy_classifier/train.spacy")

valid_docs = make_docs(random.choices(vac_train_data, k=num_texts))
doc_bin = DocBin(docs=valid_docs)
doc_bin.to_disk("./vacancy_classifier/valid.spacy")

In [38]:
!python -m spacy init fill-config ./vac_base_config.cfg ./vac_config.cfg

[+] Auto-filled config with all values
[+] Saved config
vac_config.cfg
You can now add your data and train your pipeline:
python -m spacy train vac_config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


2021-07-05 21:54:36.928505: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2021-07-05 21:54:36.928778: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [39]:
!python -m spacy train vac_config.cfg --output ./vac_output

[i] Using CPU

2021-07-05 21:54:39.933556: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2021-07-05 21:54:39.933770: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
[2021-07-05 21:54:41,953] [INFO] Set up nlp object from config
[2021-07-05 21:54:41,959] [INFO] Pipeline: ['tok2vec', 'textcat']
[2021-07-05 21:54:41,961] [INFO] Created vocabulary
[2021-07-05 21:54:41,961] [INFO] Finished initializing nlp object
[2021-07-05 21:54:50,660] [INFO] Initialized pipeline components: ['tok2vec', 'textcat']




=========================== Initializing pipeline ===========================
[+] Initialized pipeline

============================= Training pipeline =============================
[i] Pipeline: ['tok2vec', 'textcat']
[i] Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS TEXTCAT  CATS_SCORE  SCORE 
---  ------  ------------  ------------  ----------  ------
  0       0          0.00          0.25       41.95    0.42
  0     200          0.00         12.50       85.61    0.86
  0     400          0.00          6.59       91.96    0.92
  0     600          0.00          5.56       93.16    0.93
  0     800          0.00          4.40       94.01    0.94
  0    1000          0.00          2.10       94.65    0.95
  0    1200          0.00          2.98       94.66    0.95
  0    1400          0.00          2.42       95.32    0.95
  0    1600          0.00          4.34       95.38    0.95
  1    1800          0.00          1.11       95.79    0.96
  1    2000          0.00    

## 'IT' filter (featuring spacy)

In [52]:
df_it = df[['data', 'IT']]
df_it.sample(5)

,data,IT
2614,💚 Usetech приглашает в свою команду Team Lead ...,IT
795,CMO / Head of ASO\r\nв ASOdesk — платформа для...,non-IT
3113,Коммерческий директор\r\nв Малахит — компания ...,non-IT
3137,Product Manager\r\nв Реклайн— поставщик услуг ...,IT
1023,"Andrey Galaktionov, [07.04.21 17:16]\r\nПрогра...",IT


In [53]:
df_it['IT'] = df['IT'].apply(lambda x: 1 if 'IT' else 0)

<ipython-input-53-ae244801e245>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_it['IT'] = df['IT'].apply(lambda x: 1 if 'IT' else 0)


In [54]:
it_train_data = []

for i in tqdm(range(len(df_it))):
    it_train_data.append((df_it.iloc[i, 0], df_it.iloc[i, 1]))

100%|███████████████████████████████████████████████████████████████████████████| 3986/3986 [00:00<00:00, 28782.22it/s]


In [55]:
def make_it_docs(data):
    docs = []
    for doc, label in nlp.pipe(data, as_tuples = True):
        if label == 0:
            doc.cats['non-IT'] = 1
        else:
            doc.cats['IT'] = 1
        docs.append(doc)
    return (docs)

In [56]:
nlp = spacy.load('ru_core_news_sm')

In [57]:
num_texts = int(np.round(len(df_it)*0.8))
num_texts

<IPython.core.display.Javascript object>

3189

In [58]:
train_docs = make_docs(random.choices(vac_train_data, k=num_texts))
doc_bin = DocBin(docs=train_docs)
doc_bin.to_disk("./it_classifier/train.spacy")

valid_docs = make_docs(random.choices(vac_train_data, k=num_texts))
doc_bin = DocBin(docs=valid_docs)
doc_bin.to_disk("./it_classifier/valid.spacy")

In [59]:
!python -m spacy init fill-config ./it_base_config.cfg ./it_config.cfg

[+] Auto-filled config with all values
[+] Saved config
it_config.cfg
You can now add your data and train your pipeline:
python -m spacy train it_config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


2021-07-05 22:17:41.185362: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2021-07-05 22:17:41.185607: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [60]:
!python -m spacy train it_config.cfg --output ./it_output

[+] Created output directory: it_output

2021-07-05 22:17:44.252139: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2021-07-05 22:17:44.252334: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
[2021-07-05 22:17:46,358] [INFO] Set up nlp object from config
[2021-07-05 22:17:46,364] [INFO] Pipeline: ['tok2vec', 'textcat']
[2021-07-05 22:17:46,366] [INFO] Created vocabulary
[2021-07-05 22:17:46,366] [INFO] Finished initializing nlp object
[2021-07-05 22:17:55,220] [INFO] Initialized pipeline components: ['tok2vec', 'textcat']



[i] Using CPU

=========================== Initializing pipeline ===========================
[+] Initialized pipeline

============================= Training pipeline =============================
[i] Pipeline: ['tok2vec', 'textcat']
[i] Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS TEXTCAT  CATS_SCORE  SCORE 
---  ------  ------------  ------------  ----------  ------
  0       0          0.00          0.25       46.20    0.46
  0     200          0.00          8.85       47.99    0.48
  0     400          0.00          9.48       61.11    0.61
  0     600          0.00          6.39       67.87    0.68
  0     800          0.00          4.84       66.61    0.67
  0    1000          0.00          5.95       64.43    0.64
  0    1200          0.00          7.11       68.92    0.69
  0    1400          0.00          3.12       68.60    0.69
  0    1600          0.00          6.19       68.99    0.69
  1    1800          0.00          2.85       69.25    0.69
  1    2000    

#### Data labelling for IT filtering

In [61]:
df_it = df[df['vacancy']!=0]

In [63]:
df_it.sample(5)

,data,label,vacancy,IT
535,#вакансия #Москва #разработчик #fullstack\r\nП...,Full Stack разработчик,1,IT
326,#вакансия \n\nКомпания: Smartbrain\n\nВилка 30...,unknown,1,IT
139,#вакансия #Москва #scala #spark #SCALA #Spring...,Java-разработчик,1,IT
2997,#вакансия #Санкт-Петербург\r\n\r\nГруппа компа...,PHP-разработчик,1,IT
1927,Senior Python Developer/Team Lead\r\nв ГдеМате...,Python-разработчик,1,IT


In [64]:
df_it = df_it[['data', 'IT']]

In [67]:
from IPython.display import clear_output

In [ ]:
### 1 if IT, 0 if not

for i in range(len(df_it)):
    print(i, 'out of', len(df_it))
    df_it.iloc[i, 1] = input(df_it.iloc[i, 0]+'\n')
    clear_output()

312 out of 2867
